In [ ]:
import re
import string
import nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize , sent_tokenize
from nltk.corpus import wordnet

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!pip install contractions
import contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 13.3 MB/s eta 0:00:00


In [ ]:
def contraction_remover(text):
    expanded_words = []
    for word in text.split():
    # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_words)

In [ ]:
def contraction_remover(text):
    expanded_words = []
    for word in text.split():
    # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_words)


def normalize_corpus(df):
    # Remove HTML tags
    df['Preprocess_Article'] = df['Article'].apply(lambda x: re.sub(r'<.*?>', '', x))

    # Convert to lowercase
    df['Preprocess_Article'] = df['Preprocess_Article'].str.lower()

    # Remove URLs
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: re.sub(r'http\S+|www\.\S+', '', x))

    # Remove email addresses
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: re.sub(r'\S+@\S+', '', x))

    # Remove phone numbers
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: re.sub(r'\d{10}', '', x))

    # Handle negation
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: re.sub(r'\bnot\b(\w+)', r'not_\1', x))

    # Remove special characters and punctuation
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

    # Remove numeric tokens
    df['Preprocess_Article'] = df['Preprocess_Article'].apply(lambda x: re.sub(r'\b\d+\b', '', x))

    # Tokenization
    df['tokens'] = df['Preprocess_Article'].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

    # Stemming
    stemmer = PorterStemmer()
    df['tokens'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word,get_wordnet_pos(word))for word in x])

     # Convert tokens into a single string
    df['Clean Article'] = df['tokens'].apply(lambda x: ' '.join(x))

    return df

In [ ]:
# Helper function to map POS tag to WordNet POS tag
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# Fetch data
data=fetch_20newsgroups(subset='all')
data = fetch_20newsgroups(subset='all', shuffle=True,remove=('headers', 'footers', 'quotes'))
data_labels_map = dict(enumerate(data.target_names))

In [ ]:
# Create objects for each package
import numpy as np
#import text_normalizer as tn
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# building the dataframe for the data extracted from newgroups
# Create a corpus of newsgroup sentences
corpus=data.data
target_labels=data.target
target_names = [data_labels_map[label] for label in data.target]
data_df = pd.DataFrame({'Article': corpus, 'Target Label': target_labels,'Target Name': target_names})
print(data_df.shape)
data_df.head(10)

(18846, 3)


,Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,19,talk.religion.misc


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18846 entries, 0 to 18845
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Article       18846 non-null  object
 1   Target Label  18846 non-null  int64 
 2   Target Name   18846 non-null  object
dtypes: int64(1), object(2)
memory usage: 441.8+ KB


In [ ]:
normalize_corpus(data_df)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18846 entries, 0 to 18845
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Article             18846 non-null  object
 1   Target Label        18846 non-null  int64 
 2   Target Name         18846 non-null  object
 3   Preprocess_Article  18846 non-null  object
 4   tokens              18846 non-null  object
 5   Clean Article       18846 non-null  object
dtypes: int64(1), object(5)
memory usage: 883.5+ KB


In [ ]:
# view sample data
data_df = data_df[['Article', 'Clean Article', 'Target Label', 'Target Name']]
data_df.head(10)

,Article,Clean Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,sure basher pen fan pretti confus lack kind po...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,brother market highperform video card support ...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,final say dream mediterranean new area great y...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think scsi card dma transfer disk scsi card dm...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,old jasmin drive use new system understand ups...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,back high school work lab assist bunch experim...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,ae dallastri tech support may line one get start,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",stuff delet ok here solut problem move canada ...,10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",yeah second one believ price ive tri get good ...,10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,christian mean someon believ divin jesu safe s...,19,talk.religion.misc


In [ ]:
data_df = data_df.replace(r'^(\s?)+$', np.nan, regex=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18846 entries, 0 to 18845
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Article        18331 non-null  object
 1   Clean Article  18296 non-null  object
 2   Target Label   18846 non-null  int64 
 3   Target Name    18846 non-null  object
dtypes: int64(1), object(3)
memory usage: 589.1+ KB


In [ ]:
data_df = data_df.dropna().reset_index(drop=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18296 entries, 0 to 18295
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Article        18296 non-null  object
 1   Clean Article  18296 non-null  object
 2   Target Label   18296 non-null  int64 
 3   Target Name    18296 non-null  object
dtypes: int64(1), object(3)
memory usage: 571.9+ KB


In [ ]:
# Create a csv file to resue it.
data_df.to_csv('clean_newsgroups.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names = train_test_split(np.array(data_df['Clean Article']), np.array(data_df['Target Label']),
np.array(data_df['Target Name']),test_size=0.33, random_state=42)

In [ ]:
train_corpus.shape, test_corpus.shape

((12258,), (6038,))

In [ ]:
# Use Counter to create the dictionary with label names
from collections import Counter

In [ ]:
trd = dict(Counter(train_label_names))
tsd = dict(Counter(test_label_names))

In [ ]:
trd

{'comp.graphics': 648,
 'sci.med': 649,
 'sci.electronics': 647,
 'soc.religion.christian': 660,
 'comp.sys.ibm.pc.hardware': 635,
 'talk.religion.misc': 400,
 'rec.sport.baseball': 631,
 'talk.politics.guns': 584,
 'talk.politics.misc': 502,
 'alt.atheism': 490,
 'rec.motorcycles': 614,
 'comp.sys.mac.hardware': 624,
 'rec.sport.hockey': 653,
 'comp.windows.x': 662,
 'comp.os.ms-windows.misc': 623,
 'misc.forsale': 656,
 'rec.autos': 649,
 'talk.politics.mideast': 627,
 'sci.space': 639,
 'sci.crypt': 665}

In [ ]:
(pd.DataFrame([[key, trd[key], tsd[key]] for key in trd],columns=['Target Label', 'Train Count', 'Test Count'])
.sort_values(by=['Train Count', 'Test Count'],ascending=False))

,Target Label,Train Count,Test Count
19,sci.crypt,665,296
13,comp.windows.x,662,318
3,soc.religion.christian,660,314
15,misc.forsale,656,303
12,rec.sport.hockey,653,319
1,sci.med,649,311
16,rec.autos,649,285
0,comp.graphics,648,305
2,sci.electronics,647,309
18,sci.space,639,314


from matplotlib import pyplot as plt
_df_0['Train Count'].plot(kind='hist', bins=20, title='Train Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Test Count'].plot(kind='hist', bins=20, title='Test Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='Train Count', y='Test Count', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Train Count'].plot(kind='line', figsize=(8, 4), title='Train Count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_4['Test Count'].plot(kind='line', figsize=(8, 4), title='Test Count')
plt.gca().spines[['top', 'right']].set_visible(False)